# From Deep Learning Foundations to Stable Diffusion

### Highlights for study:
#### Lecture 2
- Path object
- generators/islice -> use generator instead of lists
- \_\_getitem\_\_(self, idxs) -> [] overload
- map -> apply f to iterable
- globality of random state
- torch.unique; torch.bincount

#### Lecture 3
- partial (funcs with partial input args), equivalent to 2 lambdas (lect5c)

#### Lecture 4
- nn.Module

#### Lecture 5a
- hasattr; works with class members/methods
- \_\_setattr\_\_ -> creates attribute dynamically
- reduce
- \_\_iter\_\_ + next(iter(x))
- yield from 
- dataset
- collate_fn
- dataloader

#### Lecture 5b
- @inplace
- itemgetter
- ig = itemgetter('a', 'c') # returns a func that gets vals form dict at keys 'a' and 'c'; call: ig(dict)
- default_collate: input: 2 (or more) dicts with k keys -> returns {k:k, v: stacks together values for same keys across input dicts}

#### Lecture 5c
- callbacks: a callable that will calld back to when smthing happens
- partial (funcs with partial input args), equivalent to 2 lambdas
- Callable Classes: \_\_call\_\_ used to store state/info across calls
- getattr(self.cb,cb_name, None) # from callback obj select method
- \_\_getattr\_\_(self, k,v) used when call: foo.x: k=x, v=foo.x; it is only called if the member does not exists 

#### Lecture6
- [[(i,j) for j in range(N)] for i in range(N)] # structs coords grid
- to_device

#### Lecture7
- @classmethod, alternative ctor
- fc.store_attr()
- @property -> calls method without parenthesis, the method must do not take inputs (only self)

#### Lecture8
- torch metrics: https://torchmetrics.readthedocs.io/en/stable/pages/quickstart.html
- detach
- ...todo

#### Lecture8b
- decorators detail
- callbacks
- @property
- lrfinder
- momentum

#### Lecture8c
- model diagnostic
- lr idea/analysis on its abs value
- scheduler

#### Lecture9a INITIALIZATION IMPO
- model diagnostic
- lr idea/analysis on its abs value
- IMPO: on w init and normalizations
- IMPO: reasoning on act func generalRelu
- You need to init your net correctly wrt your activation function!

#### Lecture9b
- optimizers momentum rmsprop adam  
- api exploration 
- schedulers
- OneCycleLR

# Dataset class

In [ ]:
class Dataset():
    def __init__(self, x, y):
        self.x, self.y = x, y
        
    def __len__(self): 
        return len(self.x)
    
    def __getitem__(self, i): # overloader for [] operator, supports slicing
        # preprocessing, usually is parallelized
        # dont send anything to device here cuz huge overload if multiple worker
        return self.x[i], self.y[i]

# Collate fnc (optionally) used in DataLoader

In [ ]:
import torch
def collate(b): # b = [(xi,yi), (xj,yj), ...]
    # preprocessing, usually is parallelized
    # dont send anything to device here cuz huge overload if multiple worker
    xs, ys = zip(*b) # creates 2 lists with all xis and all yis
    return torch.stack(xs), torch.stack(ys)

# DataLoader

In [ ]:
from torch.utils.data import DataLoader
bs = bs if train else if valid 2*bs
shuffle = True if train if valid False

dl = DataLoader(dataset_obj, bs, shuffle=shuffle, drop_last=?, num_workers, collate_fn = collate_fn)  

# Decorators and inplace decorator 

In [ ]:
# to make an implace func have a return 
def inplace(f): # input: a func, returns a func
    def _f(b): # defs a new func
        f(b) # applies the input func that acts on its input inplace
        return b # return the modified input
    return _f # return a func that modifies b inplace

# @ before a func: take the func defd here below, pass it as input to the @func, replace the func defd here below with the func returned by the @func
@inplace # name of the decorating func
def transform_(b): 
    b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]
    

# ToDevice

In [ ]:
device = "mps" if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

from typing import Mapping
def to_device(x, device=device): 
    if isinstance(x, Mapping): # i.e. if it is a dict -> if isinstance(x, dict):
        return {k:v.to(device) for k,v in x.items()} # sends each val to device
    return type(x)(o.to(device) for o in x)    

'''
Mapping: A container object that supports arbitrary key lookups and implements the methods 
    specified in the collections.abc.Mapping or collections.abc.MutableMapping abstract base classes. 
    Examples include dict, collections.defaultdict, collections.OrderedDict and collections.Counter.
'''

'''
type(x)(...): Creates a new object of the same type as x. 
    It's equivalent to calling the constructor of the type with the arguments inside the parentheses.
'''